In [ ]:
from autogluon.tabular import TabularPredictor
import pandas as pd
from Utils.dataManagingUtils import save_prediction_to_file
import nbimporter
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import TransformerMixin, BaseEstimator   
import numpy as np
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import numpy as np
from Utils.dataManagingUtils import save_prediction_to_file
from Utils.dataManagingUtils import save_data_to_csv
from datetime import datetime
from Utils.transformersUtils import transformers


In [ ]:
label_name = 'label'
X_train = pd.read_csv('..\\Dane\\Oryginalne\\artificial_train.data', header=None, sep=' ').iloc[:, :500]
y_train = pd.read_csv('..\\Dane\\Oryginalne\\artificial_train_labels.data', header=None, sep=' ', names=[label_name]).iloc[:, 0]
X_test = pd.read_csv('..\\Dane\\Oryginalne\\artificial_test.data', header=None, sep=' ').iloc[:, :500]

## AutoGluon

Zbudowanie modelu za pomocą frameworku AutoGluon wymagało dostosowania do zastosowanych metod preprocessingu. W celu posłużenia się TabularPredictorem w pipelinie konieczne było zaimplementowanie pomocniczej klasy implementującej metody `fit` oraz `transform`. W metodzie `fit` następuje odpowiednie przygotowanie ramki danych, stworzenie `TabularPredictor` z metryką `balanced_accuracy` oraz wytrenowanie modelu. Zależało nam na jak najlepszej jakości modelu, więc zastosowano `best_quality` jako zestaw konfiguracyjny. Natomiast metoda `transform` została wykorzystana do wydobycia odpowiednich danych z prognozy prawdopodobieństw przynależności do klas oraz informacji o wytrenowanych modelach i ich jakości.

In [ ]:
class AutoGluonClassifier(BaseEstimator, TransformerMixin):
    def __init__(self, label_name):
        self.predictor = None
        self.label_name = label_name
    
    def fit(self, X, y):
        train_data = pd.concat([
            pd.DataFrame(X),
            pd.DataFrame(y)
        ], axis=1)
        self.predictor = TabularPredictor(
            label=label_name,
            eval_metric='balanced_accuracy'
        ).fit(
            train_data,
            presets='best_quality',
            verbosity=0
        )
        return self
    
    def transform(self, X):
        proba = self.predictor.predict_proba(pd.DataFrame(X))
        return proba.iloc[:, 1:], self.predictor.leaderboard(extra_info=True), self.predictor.model_best

Każda przygotowana metoda preprocessingu została zawarta w pipeline razem z klasyfikatorem autogluona. Wytrenowano modele i wydobyto z nich informacje o jakości i ich rankingu. Na danych testowych obliczono prawdopodobieństwo przynależności do klasy 1. Wszystkie potrzebne dane zapisano do plików w celu późniejszej analizy.

In [ ]:
for transformer in transformers:
    ag_classifier = AutoGluonClassifier(label_name)
    name = transformer.get_name()

    print(f'\nTransformer: {transformer.get_name()}...')
    pipeline = Pipeline([
        ('preprocessing', transformer),
        ('classifier', ag_classifier)
    ])

    pipeline.fit(X=X_train, y=y_train)
    proba, leaderboard, best_model = pipeline.transform(X=X_test)

    save_prediction_to_file(proba, f'..\\Wyniki\\Predykcje\\AUTO\\autogluon\\predictions_ag_best_{name}.txt')
    save_data_to_csv(leaderboard, f'..\\Wyniki\\Predykcje\\AUTO\\autogluon\\{name}_best.csv')
    
    print(f'Best model: {best_model}.')
    print(f"Score: {leaderboard.iloc[0]['score_val']}")
    print('Done.')


## Autosklearn 2.0

Klasyfikator frameworka autosklearn implementuje metody `fit` oraz `transform`, więc nie jest konieczna implementacja własnego klasyfikatora jak w przypadku AutoGluona.

Przygotowano odpowiednie parametry przyjmowane przez `AutoSklearn2Classifier` - metryka `balanced_accuracy` oraz liczba równolegle wykonywanych zadań w procesie optymalizacji hiperparametrów.

Każda przygotowana metoda preprocessingu została zawarta w pipeline razem z klasyfikatorem autosklearn 2.0. Wytrenowano modele i wydobyto z nich informacje o jakości i ich rankingu. Na danych testowych obliczono prawdopodobieństwo przynależności do klasy 1. Wszystkie potrzebne dane zapisano do plików w celu późniejszej analizy.

*Uwaga: poniższy kod był uruchamiany na środowisku `Google Colab`. Może nie zadziałać w tym notatniku.*

In [ ]:
from autosklearn.metrics import balanced_accuracy
from autosklearn.experimental.askl2 import AutoSklearn2Classifier

settings = {
  "metric": balanced_accuracy,
  "n_jobs": 4,
}

for transformer in transformers:
  askl2 = AutoSklearn2Classifier(**settings)
  name = transformer.get_name()

  print(f'\nTransformer: {name}...')
  pipeline = Pipeline([
      ('preprocessing', transformer),
      ('classifier', askl2)
  ])

  pipeline.fit(X=X_train, y=y_train)

  proba = pd.DataFrame(pipeline.predict_proba(X=X_test)[:, 1])
  save_prediction_to_file(proba, f'drive/MyDrive/automl/predictions_askl2_{name}')

  res = pd.DataFrame(askl2.cv_results_)
  max = askl2.cv_results_['mean_test_score'].max()
  leaderboard = askl2.leaderboard(sort_by="rank")

  print(leaderboard)

  save_data_to_csv(res, f'drive/MyDrive/automl/{name}_{max}.csv')

  print('Done.')

## mljar

Klasyfikator frameworka MLJar implementuje metody `fit` oraz `transform`, więc nie jest konieczna implementacja własnego klasyfikatora jak w przypadku AutoGluona.

Przygotowano odpowiednie parametry przyjmowane przez `AutoML` - metryka `accuracy`, maksymalny czas wykonania (1h - tyle samo ile w powyższych frameworkach) oraz liczba równolegle wykonywanych zadań w procesie optymalizacji hiperparametrów.

Każda przygotowana metoda preprocessingu została zawarta w pipeline razem z klasyfikatorem z MLJar. Wytrenowano modele i wydobyto z nich informacje o jakości i ich rankingu (MLJar generuje je automatycznie pod podaną ścieżką). Na danych testowych obliczono prawdopodobieństwo przynależności do klasy 1.

*Uwaga: poniższy kod był uruchamiany na środowisku `Google Colab`. Może nie zadziałać w tym notatniku.*

In [ ]:
from supervised.automl import AutoML # mljar-supervised
  
for transformer in transformers:
  name = transformer.get_name()
  formatted_date = datetime.now().strftime("%Y%m%d_%H%M%S")
  
  automl = AutoML(
      mode="Perform", 
      eval_metric='accuracy',
      total_time_limit=3600,
      results_path=f'drive/MyDrive/automl/AutoML_{name}_{formatted_date}',
      n_jobs=4
  )

  print(f'\nTransformer: {name}...')
  pipeline = Pipeline([
      ('preprocessing', transformer),
      ('classifier', automl)
  ])
  automl.verbose_print
  pipeline.fit(X_train, y_train)

  proba = pipeline.predict_proba(X_test)

  save_prediction_to_file(proba, f'drive/MyDrive/automl/predictions_mljar_{name}')